# Tensorboard Example using Simple MNIST

This tutorial will show you how to get started with Tensorboard.
Tensorboard is a visualization tool that you can use along with Tensorflow. You can do a lot of different things with Tensorboard, such as plot metrics about the execution of your (computational) graphs, visualize images that pass through your graphs, etc.

In brief, the steps to use Tensorboard is as follow:
1. Indicate which nodes in your program from which you would like to collect the summary of data you want to visualize
2. Within a running session, generate and merge all the summary nodes, by using tf.summary.merge_all
3. Write the merged summary to disk, using tf.summary.FileWriter
4. Access the Tensorboard to view the summary

We will go through the four steps above in the following tutorial, using a simple learning on MNIST dataset. 

This tutorial is based on *https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/4_Utils/tensorboard_basic.ipynb 


In [3]:
import tensorflow as tf

If you haven't already, download the MNIST dataset.

In [9]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Declare a directory to which your graph summaries will be saved. 

In [4]:
logs_path = '/tmp/tensorflow_logs/'

In the following part, we will declare the fixed parameters and input parameters for our learning. 
This part is the same as the previous examples on declaring parameters for learning.

In [5]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

After declaring our fixed parameters, we would continue with declaring our learning method: the model we would learn, its cost function, its optimizer and accuracy measure.

Below, you see that we declare our nodes/ops into scopes. A scope can contain more than one node/op. Having a scope would particularly help when you want to visualize your network. 
This is not a scope of this tutorial, but you can learn more about visualizing your network here: https://www.tensorflow.org/get_started/graph_viz

In [6]:
# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In the following section, we declare which nodes we would like to summarize, and in which way. 
Depending on what you want to visualize, you can choose from different types of summary ops: scalar, histogram, audio, image, etc. 
In this example, we will choose a scalar summary, as we would like to monitor our cost function in each epoch, as well as the accuracy of our learning.

In [7]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

Once we have declared the summary ops, and the merged summary Tensor, we can then continue with running our Session.

To use Tensorboard, in our running Session, we would need to create the tf.summary.Filewriter Tensor, and let it know which directory (logs_path) it should write the summary logs to.

Afterwards, we need to run our merged_summary_op along with the other ops in the learning process (optimizer, cost). Finally, the output summary is added into the log file, by using summary_writer.add_summary().

In [10]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

    print "Optimization Finished!"

    # Test model
    # Calculate accuracy
    print "Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels})


Epoch: 0001 cost= 1.182138977
Epoch: 0002 cost= 0.664631298
Epoch: 0003 cost= 0.552558806
Epoch: 0004 cost= 0.498570960
Epoch: 0005 cost= 0.465417103
Epoch: 0006 cost= 0.442526688
Epoch: 0007 cost= 0.425479051
Epoch: 0008 cost= 0.412129293
Epoch: 0009 cost= 0.401385706
Epoch: 0010 cost= 0.392419814
Epoch: 0011 cost= 0.384733562
Epoch: 0012 cost= 0.378202107
Epoch: 0013 cost= 0.372406618
Epoch: 0014 cost= 0.367222395
Epoch: 0015 cost= 0.362659236
Epoch: 0016 cost= 0.358608620
Epoch: 0017 cost= 0.354867672
Epoch: 0018 cost= 0.351428938
Epoch: 0019 cost= 0.348337940
Epoch: 0020 cost= 0.345400420
Epoch: 0021 cost= 0.342753728
Epoch: 0022 cost= 0.340205403
Epoch: 0023 cost= 0.337913331
Epoch: 0024 cost= 0.335709043
Epoch: 0025 cost= 0.333699787
Optimization Finished!
Accuracy: 0.9143


Up until this point, the summaries of the graphs we are interested in have been saved in a log file in the '/tmp/tensorflow_logs/' directory (or whatever path you gave before). 

To visualize it on Tensorboard, we need to first activate it through the command line, and then open it in our browser.

In [11]:
print "Run the command line:\n" \
      "--> tensorboard --logdir=/tmp/tensorflow_logs " \
      "\nThen open http://0.0.0.0:6006/ into your web browser"

Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
Then open http://0.0.0.0:6006/ into your web browser
